In [124]:
!pip install twython 
from google.colab import drive
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import pandas as pd
import warnings

In [125]:
#Mount Google drive, provide auth code to write/read from G drive
drive.mount("/drive")
warnings.filterwarnings('ignore')

#VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media. 
nltk.download('vader_lexicon')


Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [131]:
raw_data = pd.read_csv("/drive/My Drive/Colab Notebooks/vim_raw.csv") 
raw_data.loc[raw_data['selftext'].isnull(), 'selftext'] = " "
print("raw_data : ",raw_data.shape)

#filtered_data = raw_data[raw_data['selftext'].notnull()] 
filtered_data = raw_data.loc[raw_data['selftext'] != "[removed]"] #filtering the removed or deleted posts

print("filtered_data : ",filtered_data.shape)
filtered_data.head()

raw_data :  (1136, 15)
filtered_data :  (1132, 15)


,author,author_fullname,created_utc,domain,full_link,is_crosspostable,link_flair_text,num_comments,num_crossposts,over_18,permalink,score,selftext,title,total_awards_received
0,liuchengxu,t2_zrbv9,01-01-2020,self.vim,https://www.reddit.com/r/vim/comments/eiazzo/v...,True,NaN,41,0,False,/r/vim/comments/eiazzo/vimclap_to_be_a_perform...,1,"Previously, vim-clap's main feature is the ful...",vim-clap: to be a performant fuzzy finder and ...,0
1,arumoy_shome,t2_oql3e,01-01-2020,self.vim,https://www.reddit.com/r/vim/comments/eihldw/n...,True,NaN,0,0,False,/r/vim/comments/eihldw/new_plugin_vimtextlists/,1,"Hello all, recently I wrote a new vim plugin '...",New plugin: vim-text-lists,0
2,nebulaeandstars,t2_6rhhbj7,01-01-2020,self.vim,https://www.reddit.com/r/vim/comments/eij4s8/h...,True,NaN,0,0,False,/r/vim/comments/eij4s8/how_many_of_yiuhrol_kee...,1,,How many of yiuhrol keep accidentally doing th...,0
3,eliseu_videira,t2_2t2k1fqy,01-01-2020,self.vim,https://www.reddit.com/r/vim/comments/eil8pv/w...,True,NaN,24,0,False,/r/vim/comments/eil8pv/what_would_be_the_best_...,1,The task i'm trying to accomplish is to remove...,What would be the best way to format this code...,0
4,Hollow_5oul,t2_2j29u115,01-01-2020,self.vim,https://www.reddit.com/r/vim/comments/eime7c/t...,True,NaN,25,0,False,/r/vim/comments/eime7c/til_run_lines_of_code_a...,1,"I've been using vim for a couple of years, and...",TIL - Run lines of code and get the output ins...,0


In [132]:
sia = SIA()
post_sentiment_results = []
posts = filtered_data['selftext'].tolist()

for post in posts:
    pol_score = sia.polarity_scores(post)
    pol_score['post'] = post
    post_sentiment_results.append(pol_score)

df = pd.DataFrame.from_records(post_sentiment_results)

df['post_sentiment'] = "Neutral"
df.loc[df['compound'] > 0.2, 'post_sentiment'] = "Positive"
df.loc[df['compound'] < -0.2, 'post_sentiment'] = "Negative"
#print(df.head())

title_sentiment_results =[]
titles = filtered_data['title'].tolist()
for title in titles:
    pol_score = sia.polarity_scores(title)
    pol_score['title'] = title
    title_sentiment_results.append(pol_score)

df2 = pd.DataFrame.from_records(title_sentiment_results)
df2['title_sentiment'] = "Neutral"
df2.loc[df2['compound'] > 0.2, 'title_sentiment'] = "Positive"
df2.loc[df2['compound'] < -0.2, 'title_sentiment'] = "Negative"
#print(df2.head())


feature_df = df[['post', 'post_sentiment']]
feature_df[['title', 'title_sentiment']] = df2[['title', 'title_sentiment']].to_numpy()
feature_df['reddit_score'] = filtered_data['score'].values
feature_df['num_comments'] = filtered_data['num_comments'].values
feature_df['date_created'] = filtered_data['created_utc'].values
feature_df['author'] = filtered_data['author'].values

feature_df = feature_df.reindex(columns=['post','title','date_created','author','post_sentiment','title_sentiment','reddit_score','num_comments'])
print(feature_df.shape)
feature_df.head()

(1132, 8)


,post,title,date_created,author,post_sentiment,title_sentiment,reddit_score,num_comments
0,"Previously, vim-clap's main feature is the ful...",vim-clap: to be a performant fuzzy finder and ...,01-01-2020,liuchengxu,Positive,Neutral,1,41
1,"Hello all, recently I wrote a new vim plugin '...",New plugin: vim-text-lists,01-01-2020,arumoy_shome,Positive,Neutral,1,0
2,,How many of yiuhrol keep accidentally doing th...,01-01-2020,nebulaeandstars,Neutral,Negative,1,0
3,The task i'm trying to accomplish is to remove...,What would be the best way to format this code...,01-01-2020,eliseu_videira,Positive,Positive,1,24
4,"I've been using vim for a couple of years, and...",TIL - Run lines of code and get the output ins...,01-01-2020,Hollow_5oul,Positive,Neutral,1,25


In [133]:
feature_df.to_csv('/drive/My Drive/Colab Notebooks/vim_feature.csv', mode='w', encoding='utf-8', index=False)
